In [ ]:
import numpy as np
import numpy.ma as ma
import matplotlib.pyplot as plt
from pylab import rcParams
import plio
from plio.io.io_gdal import GeoDataset as GeoDataSet

rcParams['figure.figsize'] = 15, 15

In [ ]:
data_prefix = '/scratch/thermal_stuff/data/'

In [ ]:
tir = GeoDataSet(data_prefix + 'LC08_L1GT_165198_20131110_20170428_01_T2_B10_SensorRad.TIF')
tir_data = tir.read_array()

In [ ]:
plt.imshow(tir_data, cmap='inferno')
plt.colorbar()

In [ ]:
x_middle = len(tir_data) // 2
y_middle = len(tir_data[x_middle]) // 2
middle_area = [(x_middle - 15, y_middle - 15), (x_middle + 15, y_middle + 15)]

non_volc = tir_data.copy()
non_volc_min = non_volc.min()
non_volc[middle_area[0][0]:middle_area[1][0], middle_area[0][1]:middle_area[1][1]] = non_volc_min

non_volc_max = non_volc.max()
volc = tir_data[middle_area[0][0]:middle_area[1][0], middle_area[0][1]:middle_area[1][1]]

volc[15, 15] = 20.0 # artificialy add hotspot because none show up with vast on this data set

In [ ]:
def average_surrounding( volc_area , x, y ):
    total = 0
    for i in (-1,0,1):
        for j in (-1,0,1):
            if i == 0 and j ==0:
                continue
            total += volc_area[ (x + i) ,( y + j) ]
    return total / 8

In [ ]:
def vast( volc_area, non_volc_area, non_volc_area_max, middle_area ):
    anomolies = np.empty(non_volc_area.shape) 
    anomolies[:] = False
    for column in range(len(volc_area) - 1):
        for row in range(len(volc_area[column]) - 1):
            if volc_area[column, row] - average_surrounding( volc_area, column, row ) > non_volc_area_max:
                anomolies[column + middle_area[0][0], row + middle_area[0][1]] = True
    return anomolies

In [ ]:
anomolies = vast( volc, non_volc, non_volc_max, middle_area)

In [ ]:
plt.imshow(anomolies, cmap='inferno')
plt.colorbar()

In [ ]:
points = np.argwhere(anomolies==True) # should be 3855 and 3780 the middle points
np.argwhere(anomolies==True)